In [2]:
import sys
sys.path.append("../")

from fns import get_intent, messages_prompt
from tools import page_scraper, web_search

## Intention Testing

This tests whether or not the determined intent of the user has been inferred.

These tests allow benchmarking over time when new actions/tools are added that
might have an impact on previously established intent detectors.


In [3]:
from IPython.display import Markdown as md

tests = [
    {
        "query": "Add a taxpayer for me",
        "intent": "action_vtx_api",
    },
    {
        "query": "Add a new user",
        "intent": "action_vtx_api",
    },
    {
        "query": "Update a taxpayer",
        "intent": "action_vtx_api",
    },
    {
        "query": "Delete a taxpayer",
        "intent": "action_vtx_api",
    },
    {
        "query": "Please elaborate on that for me.",
        "intent": "elaborate_vtx_url",
    },
    {
        "query": "What are the features of O Series?",
        "intent": "search_vtx_kb",
    },
    {
        "query": "What products does Vertex offer?",
        "intent": "search_vtx_com",
    },
    {
        "query": "How do I contact customer service?",
        "intent": "search_vtx_com",
    },
    {
        "query": "Add a taxpayer for me with the following information: name: John Doe, address: 123 Main St, city: Springfield, state: IL, zip: 62701",
        "intent": "action_vtx_api",
    }
]


def test_intents(tests):
    results = []
    for test in tests:
        query = test["query"]
        intent = test["intent"]
        result = get_intent(query)
        results.append({
            **test,
            "inferred": result["name"],
            "pass": result["name"] == intent,
            "desc": result["description"] if "description" in result else ""
        })
    return results

# create markdown table


def create_table(results):
    table = "| Query | Expected | Actual | Test | Matched Descrption | \n"
    table += "| --- | --- | --- | --- | --- | \n"
    for result in results:
        query = result["query"]
        expected = result["intent"]
        actual = result['inferred']
        test = "✅" if result["pass"] else "❌"
        desc = result["desc"] if "desc" in result else ""
        table += f"| {query} | {expected} | {actual} | {test} | {desc} | \n"
    return table


results = test_intents(tests)
table = create_table(results)
md(table)

| Query | Expected | Actual | Test | Matched Descrption | 
| --- | --- | --- | --- | --- | 
| Add a taxpayer for me | action_vtx_api | action_vtx_api | ✅ | Executes a command on a user's behalf with specific information. I do not answer questions. | 
| Add a new user | action_vtx_api | action_vtx_api | ✅ | Executes a command on a user's behalf. | 
| Update a taxpayer | action_vtx_api | action_vtx_api | ✅ | Executes a command on a user's behalf with specific information. I do not answer questions. | 
| Delete a taxpayer | action_vtx_api | action_vtx_api | ✅ | Executes a command on a user's behalf with specific information. I do not answer questions. | 
| Please elaborate on that for me. | elaborate_vtx_url | elaborate_vtx_url | ✅ | Can elaborate on a webpage (for a URL). Not for summarization. | 
| What are the features of O Series? | search_vtx_kb | search_vtx_kb | ✅ | Search for specific/esoteric information about a Vertex product/service. | 
| What products does Vertex offer? | search_vtx_com | search_vtx_kb | ❌ | Search for specific/esoteric information about a Vertex product/service. | 
| How do I contact customer service? | search_vtx_com | search_vtx_com | ✅ | Questions about sales and marketing materials. | 
| Add a taxpayer for me with the following information: name: John Doe, address: 123 Main St, city: Springfield, state: IL, zip: 62701 | action_vtx_api | action_vtx_api | ✅ | Executes a command on a user's behalf with specific information. I do not answer questions. | 


In [4]:
# Data First
from glom import glom
from tools.page_scraper import page_scraper

history = [
    {
        "role": "user",
        "content": "How do I add a taxpayer in O Series Cloud?",
        "metadata": {
            "caller": "user"
        }
    },
    {
        "role": "assistant",
        "content": "Here's a chunk I found",
        "metadata": {
            "caller": "search_vtx_kb",
            "payload": [
                {
                    "title": "How to add a taxpayer in O Series Cloud",
                    "url": "https://www.vertexinc.com",
                    "heading": "Adding a Taxpayer in O Series Cloud"
                }
            ]
        }
    },
    {
        "role": "user",
        "content": "Elaborate",
        "metadata": {
            "caller": "user"
        }
    }
]

caller_history = ["user", "search_vtx_kb", "user"]

intent = {
    'name': 'elaborate_vtx_url',
    'function': page_scraper,
    'description': 'User asks me how to do execute a specific command.',
    'toolchain': [  # resolve in order (todos)
        {
            'source': 'search_vtx_kb',  # check if already in caller_history
            'grab': '0.url'
        }
    ],
    'embed': ['... 1536 dimensions ...']
}

toolchain = intent["toolchain"]

fn = intent['function']

new_history = history[:]

# if already in caller_history, grab the index of the item in the caller_history
# for each todo...
#for tool in toolchain:
#    caller = tool["source"]
#    lens = tool["grab"]
#    # get the last matching payload
#    payload = next((
#        h["metadata"]["payload"] for h in reversed(new_history)
#        if h["metadata"] and h["metadata"]["caller"] == caller
#    ), None)

#    if payload:
#        if not lens:
#            args = payload
#        else:
#            args = glom(payload, lens)
#        print(f"calling {fn.__name__} args: {args}")
#        if type(args) is dict:
#            results = fn(**args)
#            new_history.append({
#                "role": "assistant",
#                "content": results,
#                "metadata": {
#                    "caller": caller,
#                    "payload": results
#                }
#            })
#        else:
#            results = fn(args)

#        print(f"results: {results}")
#    else:
#        # must resolve the dependency myself! 🔥
#        print("no payload found")

#function_params = ["url"]

In [5]:
test = {
    "url": "https://www.vertexinc.com"
}

# grab root with glom
not ""

True

I need to break this problem down into smaller components

-   the user communicates with an LLM
-   That LLM has access to some tools
-   one of the tools is an intent processor, which determines if a tool and which is needed
-   there is a dependency resolver that checks to see if the tool needs any prerequisites and if they are met
-   if they are not met, it will ask the user for the prerequisites or try to use another tool to fulfill the requirement
-   once all the requirements are met, the tool will be used to fulfill the requirement
-   and the result will be returned to the LLM to be communicated to the user


## `strip_history_meta`: function to remove metadata from chat history


In [6]:
def strip_history_meta(history):
    """
    removes metadata from history
    """
    return [{"role": h["role"], "content": h["content"]} for h in history]

## `is_done`: Evaluator

This is a small LLM call that only determines if the job is done or not. It
doesn't do anything else.


In [7]:
# > NOTE: The prompt should include what follow-up actions are available to
# > determine if the user has what they need to complete their task.

prompt_evaluate = """
You are a helpful assistant. 
Your job is to determine if the user has what they need to complete their task.

Follow up tasks include:
- searching for more information
- asking the user for more information
- performing actions on the user's behalf

Return one of the following one-word responses:

- "done": If the user's query is addressed (they have the details they need to complete their task).

- "incomplete": If the user will need more information to complete their task.

- "error": if the result is an error message

IMPORTANT: Return only the one-word response. No additional context or
information is needed. 
"""


def is_done(history):
    """
    Decides when the assistant has fulfilled its purpose
    """
    stripped_history = strip_history_meta(history)
    # find the first user message
    user_message = next(
        (h for h in stripped_history if h["role"] == "user"),
        None
    )
    # if the user message is not found, return False
    if not user_message:
        print("Whooops! `user` message not found")
        return False
    # find the last assistant message
    assistant_message = next(
        (h for h in reversed(stripped_history) if h["role"] == "assistant"),
        None
    )
    # if the assistant message is not found, return False
    if not assistant_message:
        print("Whooops! `assistant` message not found")
        return False

    pertinent = [
        {
            "role": "system",
            "content": prompt_evaluate
        },
        user_message,
        assistant_message
    ]

    status = messages_prompt(pertinent)

    # conditions
    if status == "done":
        print("Status: the result is complete")
        return True
    elif status == "incomplete":
        print("Status: the result is incomplete")
        return False
    elif status == "error":
        print("Error: the result is an error message")
        return False
    else:
        print("Error: evaluator could not infer a valid response. Response recieved:\n", status)
        return False

## `is_done`: Tests


In [8]:
is_done_test = [
    {
        "role": "user",
        "content": "How do I add a taxpayer in O Series Cloud?"
    },
    {
        "role": "assistant",
        "content": "You can search for that information on Vertex Knowledge Base",
    },
]

add_payer_api = """
Here's how to add a taxpayer in O Series Cloud using the Vertex API:

    ```python
    import requests
    import json

    url = "https://api.vertexinc.com/taxpayer"
    headers = {
        "Authorization
    }
    payload = {
        "name": "John Doe",
        "address": "123 Main St",
        "city": "Springfield",
        "state": "IL",
        "zip": "62701"
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print(response.json())
    ```
"""

add_payer_steps = """
Here's a step-by-step guide to adding a taxpayer from the Knowledge Base:

1. Log in to Vertex O Series Cloud
2. Click on the "Taxpayers" tab
3. Click on the "Add Taxpayer" button
4. Fill in the taxpayer's information
5. Click "Save"

"""

add_payer_search_results = """
[
    { 
        title: "How to add a taxpayer in O Series Cloud", 
        url: "https://www.vertexinc.com", 
        heading: "Adding a Taxpayer in O Series Cloud" 
    }, 
    { 
        title: "Adding a taxpayer, what you need to know",
        url: "https://www.vertexinc.com/what-you-need-to-know", 
        heading: "Adding a Taxpayer"
    },
    {
        title: "Vertex O Series Cloud Documentation",
        url: "https://www.vertexinc.com/docs",
        heading: "O Series Cloud Documentation"
    }
]
"""

is_done_tests = [
    {
        "history": is_done_test,
        "expected": False,
    },
    {
        "history": [
            *is_done_test,
            {"role": "assistant",
                "content": "You can add a taxpayer in O Series Cloud using the Vertex API "}
        ],
        "expected": False,
    },
    {
        "history": [
            *is_done_test,
            {
                "role": "assistant",
                "content": add_payer_search_results
            }
        ],
        "expected": False,
    },
    {
        "history": [
            *is_done_test,
            {"role": "assistant", "content":  add_payer_api}
        ],
        "expected": True,
    },
    {
        "history": [
            *is_done_test,
            {"role": "assistant", "content":  add_payer_steps}
        ],
        "expected": True,
    }
]


def test_is_done(is_done_tests):
    results = []
    for test in is_done_tests:
        history = test["history"]
        expected = test["expected"]
        result = is_done(history)
        results.append({**test, "result": result, "pass": result == expected})
    return results


def convert_to_md_table(results):
    table = "| History | Expected | Actual | Test |\n"
    table += "| --- | --- | --- | --- |\n"
    for result in results:
        history = result["history"]
        expected = result["expected"]
        actual = result['result']
        test = "✅" if result["pass"] else "❌"
        table += f"| {history} | {expected} | {actual} | {test} |\n"
    return table


results = test_is_done(is_done_tests)
table = convert_to_md_table(results)
md(table)

Status: the result is incomplete
Status: the result is incomplete
Status: the result is incomplete
Status: the result is incomplete
Status: the result is complete


| History | Expected | Actual | Test |
| --- | --- | --- | --- |
| [{'role': 'user', 'content': 'How do I add a taxpayer in O Series Cloud?'}, {'role': 'assistant', 'content': 'You can search for that information on Vertex Knowledge Base'}] | False | False | ✅ |
| [{'role': 'user', 'content': 'How do I add a taxpayer in O Series Cloud?'}, {'role': 'assistant', 'content': 'You can search for that information on Vertex Knowledge Base'}, {'role': 'assistant', 'content': 'You can add a taxpayer in O Series Cloud using the Vertex API '}] | False | False | ✅ |
| [{'role': 'user', 'content': 'How do I add a taxpayer in O Series Cloud?'}, {'role': 'assistant', 'content': 'You can search for that information on Vertex Knowledge Base'}, {'role': 'assistant', 'content': '\n[\n    { \n        title: "How to add a taxpayer in O Series Cloud", \n        url: "https://www.vertexinc.com", \n        heading: "Adding a Taxpayer in O Series Cloud" \n    }, \n    { \n        title: "Adding a taxpayer, what you need to know",\n        url: "https://www.vertexinc.com/what-you-need-to-know", \n        heading: "Adding a Taxpayer"\n    },\n    {\n        title: "Vertex O Series Cloud Documentation",\n        url: "https://www.vertexinc.com/docs",\n        heading: "O Series Cloud Documentation"\n    }\n]\n'}] | False | False | ✅ |
| [{'role': 'user', 'content': 'How do I add a taxpayer in O Series Cloud?'}, {'role': 'assistant', 'content': 'You can search for that information on Vertex Knowledge Base'}, {'role': 'assistant', 'content': '\nHere\'s how to add a taxpayer in O Series Cloud using the Vertex API:\n\n    ```python\n    import requests\n    import json\n\n    url = "https://api.vertexinc.com/taxpayer"\n    headers = {\n        "Authorization\n    }\n    payload = {\n        "name": "John Doe",\n        "address": "123 Main St",\n        "city": "Springfield",\n        "state": "IL",\n        "zip": "62701"\n    }\n    response = requests.post(url, headers=headers, data=json.dumps(payload))\n    print(response.json())\n    ```\n'}] | True | False | ❌ |
| [{'role': 'user', 'content': 'How do I add a taxpayer in O Series Cloud?'}, {'role': 'assistant', 'content': 'You can search for that information on Vertex Knowledge Base'}, {'role': 'assistant', 'content': '\nHere\'s a step-by-step guide to adding a taxpayer from the Knowledge Base:\n\n1. Log in to Vertex O Series Cloud\n2. Click on the "Taxpayers" tab\n3. Click on the "Add Taxpayer" button\n4. Fill in the taxpayer\'s information\n5. Click "Save"\n\n'}] | True | True | ✅ |
